In [ ]:
import sys
sys.path.append("../")

from utils.utils import plot_simulated_meshgrid, plot_collage
import model
import gt_sampling

import numpy as np
import torch
import bbobtorch

In [ ]:
# create ground truth functions
n_dim = 2
samples = 1000
seed = 42

torch.manual_seed(seed)
problem_f01 = bbobtorch.create_f01(n_dim, seed=seed)
# sampling from groundtruth functions
sample_f01 = gt_sampling.get_sample(problem_f01, n_samples=samples, n_dim=2, seed=42, method='random')
X_input = sample_f01[0]

## Simulate the function with NN

In [ ]:
input_dim = 2
hidden_dim = 16
hidden_layers = 8
output_dim = 1

learning_rate = 1e-4
num_epochs = 1000

m = model.NN1(input_dim, hidden_dim, hidden_layers)
optimizer = torch.optim.SGD(m.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
y = bbobtorch.create_f01(2)(X_input)
mean, std = y.mean(), y.std()
# Training loop
for epoch in range(num_epochs):
    preds = m(X_input)
    preds = preds * std + mean
    Karloss = criterion(preds.flatten(), y.clone().detach().flatten())

    # Backward and optimize
    Karloss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {Karloss.item():.4f}')

In [ ]:
# generate mesh grid for plotting
with torch.no_grad():
    x = np.linspace(-5.0, 5.0, 100)
    y = np.linspace(-5.0, 5.0, 100)
    X, Y = np.meshgrid(x, y)
    mesh_samples = np.c_[X.ravel(), Y.ravel()]
    mesh_samples_tensor = torch.tensor(mesh_samples, dtype=torch.float32)
    mesh_results = m(mesh_samples_tensor).reshape(X.shape)*std + mean
    mesh_results = mesh_results.numpy()[:, ::-1]

In [ ]:
plot_simulated_meshgrid(X, -Y, mesh_results, model='NN')

In [ ]:
plot_collage(sample_f01[0].detach().numpy(), sample_f01[1].detach().numpy(), problem_f01, "BBOB F24", "MSE-Simulator", X, -Y, mesh_results);

In [ ]:
sys.path.append("../")
torch.save(m.state_dict(), "../models/f01_mse_nn_model.pt")

In [ ]:
# create ground truth functions
n_dim = 2
samples = 1000
seed = 42

torch.manual_seed(seed)
problem_f03 = bbobtorch.create_f03(n_dim, seed=seed)
# sampling from groundtruth functions
sample_f03 = gt_sampling.get_sample(problem_f03, n_samples=samples, n_dim=2, seed=42, method='random')
X_input = sample_f03[0]

## Simulate the function with NN

In [ ]:
input_dim = 2
hidden_dim = 16
hidden_layers = 8
output_dim = 1

learning_rate = 1e-4
num_epochs = 1000

m = model.NN1(input_dim, hidden_dim, hidden_layers)
optimizer = torch.optim.SGD(m.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
y = bbobtorch.create_f03(2)(X_input)
mean, std = y.mean(), y.std()
# Training loop
for epoch in range(num_epochs):
    preds = m(X_input)
    preds = preds * std + mean
    Karloss = criterion(preds.flatten(), y.clone().detach().flatten())

    # Backward and optimize
    Karloss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {Karloss.item():.4f}')

In [ ]:
# generate mesh grid for plotting
with torch.no_grad():
    x = np.linspace(-5.0, 5.0, 100)
    y = np.linspace(-5.0, 5.0, 100)
    X, Y = np.meshgrid(x, y)
    mesh_samples = np.c_[X.ravel(), Y.ravel()]
    mesh_samples_tensor = torch.tensor(mesh_samples, dtype=torch.float32)
    mesh_results = m(mesh_samples_tensor).reshape(X.shape)*std + mean
    mesh_results = mesh_results.numpy()[:, ::-1]

In [ ]:
plot_simulated_meshgrid(X, -Y, mesh_results, model='NN')

In [ ]:
plot_collage(sample_f01[0].detach().numpy(), sample_f01[1].detach().numpy(), problem_f01, "BBOB F03", "MSE-Simulator", X, -Y, mesh_results);

In [ ]:
sys.path.append("../")
torch.save(m.state_dict(), "../models/f03_mse_nn_model.pt")

In [ ]:
# create ground truth functions
n_dim = 2
samples = 1000
seed = 42

torch.manual_seed(seed)
problem_f24 = bbobtorch.create_f24(n_dim, seed=seed)
# sampling from groundtruth functions
sample_f24 = gt_sampling.get_sample(problem_f24, n_samples=samples, n_dim=2, seed=42, method='random')
X_input = sample_f24[0]

## Simulate the function with NN

In [ ]:
input_dim = 2
hidden_dim = 16
hidden_layers = 8
output_dim = 1

learning_rate = 1e-4
num_epochs = 1000

m = model.NN1(input_dim, hidden_dim, hidden_layers)
optimizer = torch.optim.SGD(m.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
y = bbobtorch.create_f24(2)(X_input)
mean, std = y.mean(), y.std()
# Training loop
for epoch in range(num_epochs):
    preds = m(X_input)
    preds = preds * std + mean
    Karloss = criterion(preds.flatten(), y.clone().detach().flatten())

    # Backward and optimize
    Karloss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {Karloss.item():.4f}')

In [ ]:
# generate mesh grid for plotting
with torch.no_grad():
    x = np.linspace(-5.0, 5.0, 100)
    y = np.linspace(-5.0, 5.0, 100)
    X, Y = np.meshgrid(x, y)
    mesh_samples = np.c_[X.ravel(), Y.ravel()]
    mesh_samples_tensor = torch.tensor(mesh_samples, dtype=torch.float32)
    mesh_results = m(mesh_samples_tensor).reshape(X.shape)*std + mean
    mesh_results = mesh_results.numpy()[:, ::-1]

In [ ]:
plot_simulated_meshgrid(X, -Y, mesh_results, model='NN')

In [ ]:
plot_collage(sample_f01[0].detach().numpy(), sample_f01[1].detach().numpy(), problem_f01, "BBOB F24", "MSE-Simulator", X, -Y, mesh_results);

In [ ]:
sys.path.append("../")
torch.save(m.state_dict(), "../models/f24_mse_nn_model.pt")